<a href="https://colab.research.google.com/github/arnav13081994/ReinforcementLearning/blob/main/lunary_lander_my_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [9]:
# Build the AI
import random
import numpy as np
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import deque

# Build the Network
class NeuralNet(nn.Module):
  def __init__(self, state_size, action_size, seed=42):
    super().__init__()
    self.seed = T.manual_seed(seed)
    # 3 fully connected layers
    self.fc1 = nn.Linear(state_size, 64)
    self.fc2 = nn.Linear(64, 64)
    self.fc3 = nn.Linear(64, action_size)


  def forward(self, x):
    # forward propogate the inputs and then through the Relu Activation Function
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x


# Build the Experience Replay Buffer
class ExperienceReplayBuffer:
  def __init__(self, capacity):
    self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
    self.capacity = capacity
    self.memory = deque(maxlen=capacity)

  def push(self, experience):
    self.memory.append(experience)



  def sample(self, batch_size):
    learning_experiences = random.sample(self.memory, k = batch_size)
    states = T.from_numpy(np.vstack([experience[0] for experience in learning_experiences if experience is not None])).float().to(self.device)
    actions = T.from_numpy(np.vstack([experience[1] for experience in learning_experiences if experience is not None])).long().to(self.device)
    rewards = T.from_numpy(np.vstack([experience[2] for experience in learning_experiences if experience is not None])).float().to(self.device)
    next_states = T.from_numpy(np.vstack([experience[3] for experience in learning_experiences if experience is not None])).float().to(self.device)
    dones = T.from_numpy(np.vstack([experience[4] for experience in learning_experiences if experience is not None]).astype(np.uint8)).float().to(self.device)
    return states, next_states, actions, rewards, dones



# Build the Agent Class
class Agent:

  def __init__(self, state_size, action_size, buffer_capacity, learning_rate) -> None:
    self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
    self.state_size = state_size
    self.action_size = action_size
    self.local_qnetwork = NeuralNet(self.state_size, self.action_size)
    self.target_qnetwork = NeuralNet(self.state_size, self.action_size)
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr=learning_rate)
    self.memory = ExperienceReplayBuffer(buffer_capacity)
    self.t_step = 0


  def step(self, state, action, reward, next_state, done):
    self.memory.push((state, action, reward, next_state, done))
    self.t_step = (self.t_step+1) % 4
    if self.t_step == 0:
       if len(self.memory.memory) > minibatch_size:
        experiences = self.memory.sample(minibatch_size)
        self.learn(experiences, discount_factor)


  def act(self, state, epsilon):
    """Implements an epislon greedy action selection algorithm where the index of the
    Q value is the action"""
    state = T.from_numpy(state).float().unsqueeze(0).to(self.device)
    predicted_q_values = self.local_qnetwork(state)
    if random.random() > epsilon:
      return np.argmax(predicted_q_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))


  def learn(self, experiences, discount_factor):
    states, next_states, actions, rewards, dones = experiences
    q_target_next = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_target = rewards + discount_factor * q_target_next * (1 - dones)
    q_predicted = self.local_qnetwork(states).gather(1, actions)
    loss = F.mse_loss(q_predicted, q_target)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
    self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)

  def soft_update(self, local_model, target_model, interpolation_parameter):
      for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
        target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)






In [ ]:
import gymnasium as gym
# env = gym.make('LunarLander-v3')

env = gym.make("LunarLander-v3", continuous=False, gravity=-10.0,
               enable_wind=False, wind_power=15.0, turbulence_power=1.5)


# Initialise the Hyperparams
action_size = env.action_space.n
state_size = env.observation_space.shape[0]
buffer_capacity = int(1e5)
learning_rate = 5e-4
minibatch_size = 100
discount_factor = 0.99
interpolation_parameter = 1e-3
epsilon_starting = 1.0
epsilon_ending = 0.01
epsilon_decay = 0.995
epsilon = epsilon_starting



# Initialise the Agent
agent = Agent(state_size, action_size, buffer_capacity, learning_rate)


max_steps_per_episode = 1000
max_episodes = 2000
scores_per_batch = deque(maxlen=100)

# Train the Agent
for episode in range(1, max_episodes + 1):
  state, _ = env.reset()
  score = 0
  for step in range(max_steps_per_episode):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    score += reward
    state = next_state

    if done:
      break

  scores_per_batch.append(score)
  epsilon = max(epsilon_ending, epsilon_decay*epsilon)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_per_batch)), end = "")
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_per_batch)))
  if np.mean(scores_per_batch) >= 200.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_per_batch)))
    T.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break





# Generate the Videos


Episode 100	Average Score: -156.37
Episode 200	Average Score: -84.49
Episode 300	Average Score: 15.58
Episode 400	Average Score: 101.55
Episode 421	Average Score: 119.70

In [7]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display

def show_video_of_model(agent, env_name):
    env = gym.make("LunarLander-v3", continuous=False, gravity=-10.0,
               enable_wind=False, wind_power=15.0, turbulence_power=1.5 , render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state, epsilon)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v3')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()